<a href="https://colab.research.google.com/github/adithyakini/algo_intraday/blob/master/bhavcopy_nseindia2mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mysql.connector

     |████████████████████████████████| 11.9 MB 30.2 MB/s 
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247965 sha256=1ded07b85895d3e0bec9f457e7587430f26d63c2c2520f94e6a2be5f36f89709
  Stored in directory: /root/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
Successfully built mysql.connector


In [ ]:
# STOCKS+FOREX+FUTURES
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
# holiday list https://www.nseindia.com/products-services/currency-derivatives-timings-holidays
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2021-01-26','2021-02-19','2021-03-11','2021-03-29','2021-04-01','2021-04-02','2021-04-13','2021-04-14','2021-04-21','2021-05-13','2021-05-26','2021-07-21','2021-08-16','2021-08-19','2021-09-10','2021-10-15','2021-10-19','2021-11-05','2021-11-19']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
      os.mkdir(base+y+'/bhavcopy')
    equities_bhavcopy=requests.get('https://archives.nseindia.com/products/content/sec_bhavdata_full_'+d+m+y+'.csv', timeout=10)
    if equities_bhavcopy.status_code==200:#if the connection is successful
      open('sec_bhavdata_full_'+d+m+y+'.csv', 'wb').write(equities_bhavcopy.content)
      #reading and storing in disctionary because we need 2 columns from the MTO file deliverable and %deliverable which is not found in the bhavcopy.
      f, deldict = pd.read_csv('sec_bhavdata_full_'+d+m+y+'.csv'), {}  #reading the raw dl-ed bhav file
      def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
      f = trim(pd.read_csv('sec_bhavdata_full_'+d+m+y+'.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
      f = f[f[' SERIES'] == 'EQ'] #retaining only EQ rows and leaving out bonds,options etc
      #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
      print('sec_bhavdata_full_'+d+m+y+'.csv download...SUCCESS')
      #write a new csv, bhavcopydate as a column in the csv and get rid of the downloaded file
      f['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(f))) #add a column called bhavcopydate and then fill it with the bhavecopy date
      f = f[['SYMBOL', 'BHAVCOPYDATE', ' OPEN_PRICE', ' HIGH_PRICE', ' LOW_PRICE', ' CLOSE_PRICE', ' TTL_TRD_QNTY', ' NO_OF_TRADES',' DELIV_QTY',' DELIV_PER']]
      f = f.rename(columns={'SYMBOL':'SYMBOL', ' OPEN_PRICE':'OPEN', ' HIGH_PRICE':'HIGH', ' LOW_PRICE':'LOW', ' TTL_TRD_QNTY':'TOTTRDQTY', ' NO_OF_TRADES':'NO_OF_TRADES',' DELIV_QTY':'DELIV_QTY',' DELIV_PER':'DELIV_PER'}) #rename some of the columns to something that is easier to underastand
      f.to_csv(base+y+'/bhavcopy/'+str(nextdt.date())+'.csv', index=False)
          
      ######################################################################################
      # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
      # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
      # https://pynative.com/python-mysql-database-connection/
      d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
      #check if the path exist and connect to cloud mysql 
      #if os.path.exists(base+y):
      try:
        connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
        
        db_Info = connection.get_server_info()
        print("\nConnected to MySQL Server - version", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("\nIngesting EQUITIES bhavcopy "+'/'+y+'-'+m+'-'+d+'.csv'+" into DB....:", record)
        cursor.fast_executemany = True
        with open(base+y+'/bhavcopy/'+y+'-'+m+'-'+d+'.csv', newline='',  encoding="utf8") as csvfile:
          csvdata = csv.reader(csvfile)
          #skip the 1st row as it will be header
          next(csvdata)
          pbar = ProgressBar()
          for row in pbar(list(csvdata)):
            # Prepare SQL query to INSERT a record into the database.
            sql_stocks = "INSERT INTO bhavcopy (symbol, bhavcopydate, open, high, low, close, tottrdqty, no_of_trades, delivery, delivery_percent) \
            VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9])
            #print(sql)
            try:
              #Execute the SQL command
              cursor.execute(sql_stocks)
              #Commit your changes in the database
              connection.commit()
            except Error as e:
              cprint('-------------------------------','red')
              print("Error while connecting to MySQL", e)
              cprint('-------------------------------','red')
              connection.rollback()
              #pbar.next()
              pbar.finish()
              break
      except Error as e:
        cprint('-------------------------------','red')
        print("Error while connecting to MySQL", e)
        cprint('-------------------------------','red')
        break
      finally:
        if (connection.is_connected()):
          cursor.close()
          connection.close()




    #now lets work with the FOREX
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        forex = requests.get('https://archives.nseindia.com/archives/cd/mkt_act/cd'+d+m+y+'.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound cd'+d+m+y+'.zip..skipping','red')
        break
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
        break
    if forex.status_code==200:
        fx=open(zpath, 'wb') #open the zip file under a temp location called zpath that is defined above
        fx.write(forex.content)
        fx.close()
        z, wr = zipfile.ZipFile(zpath,'r'), nextdt.date()
        z.extractall(base+y+'/Forex') #extract the contents of the zip file to a location called forex , it extracts some 6 csv files we want only 1
        z.close()
        os.remove(zpath) #remove the zip file
        if os.path.exists(base+y+'/Forex/cf'+d+m+y+'.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/Forex/cf'+d+m+y+'.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          data = data[data['INSTRUMENT'] == 'FUTCUR'] #retaining only FUTCUR rows and leaving out other rows
          data[['INSTRUMENT', 'SYMBOL    ', 'EXP_DATE  ', 'OPEN_PRICE ', 'HI_PRICE   ', 'LO_PRICE   ', 'CLOSE_PRICE','OPEN_INT*      ','TRD_VAL           ','TRD_QTY          ','NO_OF_CONT       ','NO_OF_TRADE      ']]#the useless file has spaces in the heading.
          data = data.rename(columns={'SYMBOL    ':'SYMBOL', 'EXP_DATE  ':'EXP_DATE', 'OPEN_PRICE ':'OPEN', 'HI_PRICE   ':'HIGH', 'LO_PRICE   ':'LOW', 'CLOSE_PRICE':'CLOSE','OPEN_INT*      ':'OPEN_INT','TRD_VAL           ':'TRD_VAL','TRD_QTY          ':'TRD_QTY','NO_OF_CONT       ':'NO_OF_CONT','NO_OF_TRADE      ':'NO_OF_TRADE'}) #rename some of the columns to something that is easier to underastand
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXP_DATE'] = pd.to_datetime(data['EXP_DATE'], format = '%d/%m/%Y') #the exp_date column is in format dd/mm/yyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', index=False) #write the changes above to a new file and add an _forex to the csv file
          os.remove(base+y+'/Forex/cf'+d+m+y+'.csv')
          print('Forex bhavcopy '+ str(nextdt.date())+'_forex.csv create...SUCCESS')
          #print(data)
          #cleanup all the useless files that get extracted from the currency futures bhavcopy
          for CleanUp in glob.glob(base+y+'/Forex/*.*'): # list out the files
            if not re.match(".+forex+",CleanUp): #if the list found above contains "forex" then dont do anything, else delete
              os.remove(CleanUp) #remove files in the folder
            
          try:
            connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server version", db_Info)
            #print(">>>>>>>>>>>>>>>\n")
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FOREX bhavcopy "+'/'+y+'-'+m+'-'+d+'_forex.csv'+" into DB....:", record)
            #print(">>>>>>>>>>>>>>>\n")
            cursor.fast_executemany = True
            with open(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', newline='',  encoding="utf8") as csvfile_forex:
              csvdata_forex = csv.reader(csvfile_forex)
              #skip the 1st row as it will be header
              next(csvdata_forex)
              pbar = ProgressBar()
              for row in pbar(list(csvdata_forex)):
              #Prepare SQL query to INSERT a record into the database.
                sql_forex = "INSERT INTO forex (symbol, exp_date, open, high, low, close, open_int, trd_val, trd_qty, no_of_cont, no_of_trade, bhavcopydate) \
                VALUES ('%s','%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s');" % (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                try:
                  #Execute the SQL command
                  cursor.execute(sql_forex)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
         # finally:
        #  # create cursor
        #    cursor=connection.cursor()
        #    csvfile = requests.get('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/edit#gid=0')
        #    print(csvfile.status_code)
        #    print('Updating Banned Symbols table')
            # Insert DataFrame recrds one by one.
        #    df=pd.read_csv('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/export?format=csv&gid=0', usecols=["Stocks not allowed for MIS"])
        #    pbar = ProgressBar()
            #Prepare SQL query to INSERT a record into the database.
        #    if csvfile.status_code != 200:
              #truncate banned symbols table
        #      sql_truncate_table = "truncate banned_symbols;"
        #      cursor.execute(sql_truncate_table)
        #      connection.commit()
        #      time.sleep(10)
        #      for i,row in pbar(df.iterrows()):
        #        sql_banned_symbols = "INSERT INTO banned_symbols (symbol) VALUES ('%s');" % (row[0])
        #        try:
        #          #Execute the SQL command
        #          cursor.execute(sql_banned_symbols)
                  #Commit your changes in the database
        #          connection.commit()
        #        except Error as e:
        #          cprint('-------------------------------','red')
        #          print("Error while connecting to MySQL", e)
        #          cprint('-------------------------------','red')
        #          connection.rollback()
                  #pbar.next()
        #          pbar.finish()
        #          break
        #          connection.close()




    #now lets work with the FUTURES
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        futures_bhavcopy = requests.get('https://archives.nseindia.com/content/historical/DERIVATIVES/'+y+'/'+dmonth[m]+'/fo'+d+dmonth[m]+y+'bhav.csv.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound fo'+d+dmonth[m]+y+'bhav.csv.zip..skipping','red')
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
    if futures_bhavcopy.status_code==200:
        dload=open(zpath, 'wb')
        dload.write(futures_bhavcopy.content)
        dload.close()
        #open the downlaoded bhavcopy and extract it
        z = zipfile.ZipFile(zpath, 'r')
        z.extractall(base+y+'/futures')
        z.close()
        os.remove(zpath)
        #reading and storing in a dictionary
        f, deldict = pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
        #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
        print('fo'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS')
        # making dataframe from csv file
        if os.path.exists(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          df1 = data[data['INSTRUMENT'] == 'FUTIDX'] #dataframe of only FUTIDX rows
          df2 = data[data['INSTRUMENT'] == 'FUTSTK'] #dataframe of only FUTSTK rows
          data = pd.concat([df1,df2]) #concatennating the 2 dataframes
          data=data[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'OPEN', 'HIGH', 'LOW', 'CLOSE','SETTLE_PR','CONTRACTS','VAL_INLAKH','OPEN_INT','CHG_IN_OI']]
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXPIRY_DT'] = pd.to_datetime(data['EXPIRY_DT'], format = '%d-%b-%Y') #the exp_date column is in format dd/mmm/yyyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/futures/'+ str(nextdt.date())+'_futures.csv', index=False) #write the changes above to a new file and add an _futures to the csv file
          os.remove(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv')
          print('Futures bhavcopy '+ str(nextdt.date())+'_futures.csv create...SUCCESS')
          d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
          #check if the path exist and connect to cloud mysql 
          #if os.path.exists(base+y):
          try:
            connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server - version", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FUTURES bhavcopy "+'/'+y+'-'+m+'-'+d+'_futures.csv'+" into DB....:", record)
            cursor.fast_executemany = True
            with open(base+y+'/futures/'+y+'-'+m+'-'+d+'_futures.csv', newline='',  encoding="utf8") as csvfile:
              csvdata = csv.reader(csvfile)
              #skip the 1st row as it will be header
              next(csvdata)
              pbar = ProgressBar()
              for row in pbar(list(csvdata)):
                # Prepare SQL query to INSERT a record into the database.
                sql_stocks = "INSERT INTO futures (instrument,symbol, expiry_dt, open, high, low, close, settle_pr,contracts,val_inlakh,open_int,chg_in_oi,bhavcopydate) \
                VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s', '%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                #print(sql)
                try:
                  #Execute the SQL command
                  cursor.execute(sql_stocks)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
          finally:
            if (connection.is_connected()):
              cursor.close()
              connection.close()
if (connection.is_connected()):
  cursor.close()
  connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - All imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')

2021-10-02 is a weekend
2021-10-03 is a weekend
sec_bhavdata_full_04102021.csv download...SUCCESS

Connected to MySQL Server - version 8.0.13-4


  0% (1 of 1544) |                       | Elapsed Time: 0:00:00 ETA:   0:04:46


Ingesting EQUITIES bhavcopy /2021-10-04.csv into DB....: ('7FUybt81z6',)


100% (1544 of 1544) |####################| Elapsed Time: 0:03:22 Time:  0:03:22


-------------------------------
Error while connecting to MySQL 1292 (22007): Incorrect date value: '' for column 'bhavcopydate' at row 1
-------------------------------
Forex bhavcopy 2021-10-04_forex.csv create...SUCCESS

Connected to MySQL Server version 8.0.13-4


...

fo14OCT2021bhav.csv.zip download...SUCCESS
Futures bhavcopy 2021-10-14_futures.csv create...SUCCESS

Connected to MySQL Server - version 8.0.13-4


  0% (1 of 556) |                        | Elapsed Time: 0:00:00 ETA:   0:01:38


Ingesting FUTURES bhavcopy /2021-10-14_futures.csv into DB....: ('7FUybt81z6',)


100% (556 of 556) |######################| Elapsed Time: 0:01:37 Time:  0:01:37


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
DONE - All imports complete
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# New section

In [ ]:
#STOCKS ONLY
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2021-01-26','2021-02-19','2021-03-11','2021-03-29','2021-04-01','2021-04-02','2021-04-13','2021-04-14','2021-04-21','2021-05-13','2021-05-26','2021-07-21','2021-08-16','2021-08-19','2021-09-10','2021-10-15','2021-10-19','2021-11-05','2021-11-19']
error_days =['2020-09-28']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  elif nextdt.strftime('%Y-%m-%d') in error_days:
    cprint (nextdt.strftime('%Y-%m-%d')+' bhavcopy file is unavailable','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
      os.mkdir(base+y+'/bhavcopy')
    equities_bhavcopy=requests.get('https://archives.nseindia.com/products/content/sec_bhavdata_full_'+d+m+y+'.csv', timeout=10)  
    if equities_bhavcopy.status_code==200:#if the connection is successful
      open('sec_bhavdata_full_'+d+m+y+'.csv', 'wb').write(equities_bhavcopy.content)
      #reading and storing in disctionary because we need 2 columns from the MTO file deliverable and %deliverable which is not found in the bhavcopy.
      f, deldict = pd.read_csv('sec_bhavdata_full_'+d+m+y+'.csv'), {}  #reading the raw dl-ed bhav file
      def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
      f = trim(pd.read_csv('sec_bhavdata_full_'+d+m+y+'.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
      f = f[f[' SERIES'] == 'EQ'] #retaining only EQ rows and leaving out bonds,options etc
      #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
      print('sec_bhavdata_full_'+d+m+y+'.csv download...SUCCESS')
      print (range(len(f)))
      #write a new csv, bhavcopydate as a column in the csv and get rid of the downloaded file
      f['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(f))) #add a column called bhavcopydate and then fill it with the bhavecopy date
      f = f[['SYMBOL', 'BHAVCOPYDATE', ' OPEN_PRICE', ' HIGH_PRICE', ' LOW_PRICE', ' CLOSE_PRICE', ' TTL_TRD_QNTY', ' NO_OF_TRADES',' DELIV_QTY',' DELIV_PER']]
      f = f.rename(columns={'SYMBOL':'SYMBOL', ' OPEN_PRICE':'OPEN', ' HIGH_PRICE':'HIGH', ' LOW_PRICE':'LOW', ' TTL_TRD_QNTY':'TOTTRDQTY', ' NO_OF_TRADES':'NO_OF_TRADES',' DELIV_QTY':'DELIV_QTY',' DELIV_PER':'DELIV_PER'}) #rename some of the columns to something that is easier to underastand
      f.to_csv(base+y+'/bhavcopy/'+str(nextdt.date())+'.csv', index=False)
          
      ######################################################################################
      # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
      # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
      # https://pynative.com/python-mysql-database-connection/
      d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
      #check if the path exist and connect to cloud mysql 
      #if os.path.exists(base+y):
      try:
        #connection = mysql.connector.connect(host='johnny.heliohost.org',
        #                                  database='akini_algotrade',
        #                                user='akini',
        #                              password='Drink7up@home')
        connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
        
        db_Info = connection.get_server_info()
        print("\nConnected to MySQL Server - version", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("\nIngesting EQUITIES bhavcopy "+'/'+y+'-'+m+'-'+d+'.csv'+" into DB....:", record)
        cursor.fast_executemany = True
        with open(base+y+'/bhavcopy/'+y+'-'+m+'-'+d+'.csv', newline='',  encoding="utf8") as csvfile:
          csvdata = csv.reader(csvfile)
          #skip the 1st row as it will be header
          next(csvdata)
          pbar = ProgressBar()
          for row in pbar(list(csvdata)):
            # Prepare SQL query to INSERT a record into the database.
            sql_stocks = "INSERT INTO testbhavcopy (symbol, bhavcopydate, open, high, low, close, tottrdqty, no_of_trades, delivery, delivery_percent) \
            VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9])
            #print(sql)
            try:
              #Execute the SQL command
              cursor.execute(sql_stocks)
              #Commit your changes in the database
              connection.commit()
            except Error as e:
              cprint('-------------------------------','red')
              print("Error while connecting to MySQL", e)
              cprint('-------------------------------','red')
              connection.rollback()
              #pbar.next()
              pbar.finish()
              break
      except Error as e:
        cprint('-------------------------------','red')
        print("Error while connecting to MySQL", e)
        cprint('-------------------------------','red')
        break
      finally:
        if (connection.is_connected()):
          cursor.close()
          connection.close()
if (connection.is_connected()):
  cursor.close()
  connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - STOCKS imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')

SyntaxError: ignored

In [ ]:
#FUTURES ONLY
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2021-01-26','2021-02-19','2021-03-11','2021-03-29','2021-04-01','2021-04-02','2021-04-13','2021-04-14','2021-04-21','2021-05-13','2021-05-26','2021-07-21','2021-08-16','2021-08-19','2021-09-10','2021-10-15','2021-10-19','2021-11-05','2021-11-19']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
      os.mkdir(base+y+'/bhavcopy')
    
    #now lets work with the FUTURES
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        futures_bhavcopy = requests.get('https://archives.nseindia.com/content/historical/DERIVATIVES/'+y+'/'+dmonth[m]+'/fo'+d+dmonth[m]+y+'bhav.csv.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound fo'+d+dmonth[m]+y+'bhav.csv.zip..skipping','red')
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
    if futures_bhavcopy.status_code==200:
        dload=open(zpath, 'wb')
        dload.write(futures_bhavcopy.content)
        dload.close()
        #open the downlaoded bhavcopy and extract it
        z = zipfile.ZipFile(zpath, 'r')
        z.extractall(base+y+'/futures')
        z.close()
        os.remove(zpath)
        #reading and storing in a dictionary
        f, deldict = pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
        #cprint('cm'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS', 'red', attrs=['blink'])
        print('fo'+d+dmonth[m]+y+'bhav.csv.zip download...SUCCESS')
        # making dataframe from csv file
        if os.path.exists(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          df1 = data[data['INSTRUMENT'] == 'FUTIDX'] #dataframe of only FUTIDX rows
          df2 = data[data['INSTRUMENT'] == 'FUTSTK'] #dataframe of only FUTSTK rows
          data = pd.concat([df1,df2]) #concatennating the 2 dataframes
          data=data[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'OPEN', 'HIGH', 'LOW', 'CLOSE','SETTLE_PR','CONTRACTS','VAL_INLAKH','OPEN_INT','CHG_IN_OI']]
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXPIRY_DT'] = pd.to_datetime(data['EXPIRY_DT'], format = '%d-%b-%Y') #the exp_date column is in format dd/mmm/yyyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/futures/'+ str(nextdt.date())+'_futures.csv', index=False) #write the changes above to a new file and add an _futures to the csv file
          os.remove(base+y+'/futures/fo'+d+dmonth[m]+y+'bhav.csv')
          print('Futures bhavcopy '+ str(nextdt.date())+'_futures.csv create...SUCCESS')
                  
          ######################################################################################
          # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
          # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
          # https://pynative.com/python-mysql-database-connection/
          d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year
          #check if the path exist and connect to cloud mysql 
          #if os.path.exists(base+y):
          try:
            #connection = mysql.connector.connect(host='johnny.heliohost.org',
            #                                  database='akini_algotrade',
            #                                user='akini',
            #                              password='Drink7up@home')
            connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server - version", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FUTURES bhavcopy "+'/'+y+'-'+m+'-'+d+'_futures.csv'+" into DB....:", record)
            cursor.fast_executemany = True
            with open(base+y+'/futures/'+y+'-'+m+'-'+d+'_futures.csv', newline='',  encoding="utf8") as csvfile:
              csvdata = csv.reader(csvfile)
              #skip the 1st row as it will be header
              next(csvdata)
              pbar = ProgressBar()
              for row in pbar(list(csvdata)):
                # Prepare SQL query to INSERT a record into the database.
                sql_stocks = "INSERT INTO futures (instrument,symbol, expiry_dt, open, high, low, close, settle_pr,contracts,val_inlakh,open_int,chg_in_oi,bhavcopydate) \
                VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s', '%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                #print(sql)
                try:
                  #Execute the SQL command
                  cursor.execute(sql_stocks)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
          finally:
            if (connection.is_connected()):
              cursor.close()
              connection.close()
if (connection.is_connected()):
  cursor.close()
  connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - FUTURES imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')

fo08APR2021bhav.csv.zip download...SUCCESS
Futures bhavcopy 2021-04-08_futures.csv create...SUCCESS

Connected to MySQL Server - version 8.0.13-4


N/A% (0 of 484) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Ingesting FUTURES bhavcopy /2021-04-08_futures.csv into DB....: ('7FUybt81z6',)


100% (484 of 484) |######################| Elapsed Time: 0:01:40 Time:  0:01:40


fo09APR2021bhav.csv.zip download...SUCCESS
Futures bhavcopy 2021-04-09_futures.csv create...SUCCESS

Connected to MySQL Server - version 8.0.13-4


N/A% (0 of 484) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


Ingesting FUTURES bhavcopy /2021-04-09_futures.csv into DB....: ('7FUybt81z6',)


  4% (20 of 484) |                       | Elapsed Time: 0:00:04 ETA:   0:01:35

KeyboardInterrupt: ignored

In [ ]:
#FOREX ONLY
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv
import glob
import re
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}
holiday = ['2021-01-26','2021-02-19','2021-03-11','2021-03-29','2021-04-01','2021-04-02','2021-04-13','2021-04-14','2021-04-21','2021-05-13','2021-05-26','2021-07-21','2021-08-16','2021-08-19','2021-09-10','2021-10-15','2021-10-19','2021-11-05','2021-11-19']
# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1): #loop through all dates from the last date mentioned in the bhavcopy_date file until today.
  nextdt = lastdt+ relativedelta(days=i) #calculate the next day value
  #check if the date is a weekend or market holiday so that we can remove that from the loop , bhavcopies are not available for weekends.
  if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
    cprint (nextdt.strftime('%Y-%m-%d')+' is a weekend','grey')
  elif nextdt.strftime('%Y-%m-%d') in holiday:
    cprint (nextdt.strftime('%Y-%m-%d')+' is a Market Holiday','grey')
  else:
    d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  he date
    zpath = base+y+'/'+d+'.zip'
    if not os.path.isdir(base+y):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y)
      os.mkdir(base+y+'/Index')
      os.mkdir(base+y+'/Futures')
      os.mkdir(base+y+'/Forex')
      os.mkdir(base+y+'/bhavcopy')

#now lets work with the FOREX
    for i in range(2): #try to connect to the nseindia url to download the bhavcopy , 3 times , just incase website does not respond etc.
      #while True:
      try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}#Sometimes requests from requests.get() gets blocked by server, so solution is to make the server think the request is coming from a web browser
        forex = requests.get('https://archives.nseindia.com/archives/cd/mkt_act/cd'+d+m+y+'.zip', headers=headers, timeout=5) #get the zip file from nseindia
      except requests.exceptions.Timeout as error:
        cprint('ERROR!!! 404 filenotfound cd'+d+m+y+'.zip..skipping','red')
        break
      except requests.ConnectionError:
        cprint('No connection.. retrying','red')
        break
    if forex.status_code==200:
        fx=open(zpath, 'wb') #open the zip file under a temp location called zpath that is defined above
        fx.write(forex.content)
        fx.close()
        z, wr = zipfile.ZipFile(zpath,'r'), nextdt.date()
        z.extractall(base+y+'/Forex') #extract the contents of the zip file to a location called forex , it extracts some 6 csv files we want only 1
        z.close()
        os.remove(zpath) #remove the zip file
        if os.path.exists(base+y+'/Forex/cf'+d+m+y+'.csv'):
          def trim(dataset): #Definition for strippping whitespace
              trim = lambda x: x.strip() if type(x) is str else x
              return dataset.applymap(trim)
          # making dataframe from csv file
          data = trim(pd.read_csv(base+y+'/Forex/cf'+d+m+y+'.csv'))  #reading the raw dl-ed file and trimming the trailing spaces with trim()
          data = data[data['INSTRUMENT'] == 'FUTCUR'] #retaining only FUTCUR rows and leaving out other rows
          data[['INSTRUMENT', 'SYMBOL    ', 'EXP_DATE  ', 'OPEN_PRICE ', 'HI_PRICE   ', 'LO_PRICE   ', 'CLOSE_PRICE','OPEN_INT*      ','TRD_VAL           ','TRD_QTY          ','NO_OF_CONT       ','NO_OF_TRADE      ']]#the useless file has spaces in the heading.
          data = data.rename(columns={'SYMBOL    ':'SYMBOL', 'EXP_DATE  ':'EXP_DATE', 'OPEN_PRICE ':'OPEN', 'HI_PRICE   ':'HIGH', 'LO_PRICE   ':'LOW', 'CLOSE_PRICE':'CLOSE','OPEN_INT*      ':'OPEN_INT','TRD_VAL           ':'TRD_VAL','TRD_QTY          ':'TRD_QTY','NO_OF_CONT       ':'NO_OF_CONT','NO_OF_TRADE      ':'NO_OF_TRADE'}) #rename some of the columns to something that is easier to underastand
          data['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(data))) #add a column called bhavcopydate and then fill it with the bhavecopy date
          data['EXP_DATE'] = pd.to_datetime(data['EXP_DATE'], format = '%d/%m/%Y') #the exp_date column is in format dd/mm/yyy convert it to yyyy-mm-dd
          data.to_csv(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', index=False) #write the changes above to a new file and add an _forex to the csv file
          os.remove(base+y+'/Forex/cf'+d+m+y+'.csv')
          print('Forex bhavcopy '+ str(nextdt.date())+'_forex.csv create...SUCCESS')
          #print(data)
          #cleanup all the useless files that get extracted from the currency futures bhavcopy
          for CleanUp in glob.glob(base+y+'/Forex/*.*'): # list out the files
            if not re.match(".+forex+",CleanUp): #if the list found above contains "forex" then dont do anything, else delete
              os.remove(CleanUp) #remove files in the folder
            
          try:
            #connection = mysql.connector.connect(host='johnny.heliohost.org',
            #                                  database='akini_algotrade',
            #                                user='akini',
            #                              password='Drink7up@home')
            connection = mysql.connector.connect(host='remotemysql.com',
                                            database='7FUybt81z6',
                                            user='7FUybt81z6',
                                            password='cvn1KKOijU')
            
            db_Info = connection.get_server_info()
            print("\nConnected to MySQL Server version", db_Info)
            #print(">>>>>>>>>>>>>>>\n")
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("\nIngesting FOREX bhavcopy "+'/'+y+'-'+m+'-'+d+'_forex.csv'+" into DB....:", record)
            #print(">>>>>>>>>>>>>>>\n")
            cursor.fast_executemany = True
            with open(base+y+'/Forex/'+ str(nextdt.date())+'_forex.csv', newline='',  encoding="utf8") as csvfile_forex:
              csvdata_forex = csv.reader(csvfile_forex)
              #skip the 1st row as it will be header
              next(csvdata_forex)
              pbar = ProgressBar()
              for row in pbar(list(csvdata_forex)):
              #Prepare SQL query to INSERT a record into the database.
                sql_forex = "INSERT INTO forex (symbol, exp_date, open, high, low, close, open_int, trd_val, trd_qty, no_of_cont, no_of_trade, bhavcopydate) \
                VALUES ('%s','%s','%s', '%s','%s', '%s', '%s', '%s','%s','%s', '%s','%s');" % (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12])
                try:
                  #Execute the SQL command
                  cursor.execute(sql_forex)
                  #Commit your changes in the database
                  connection.commit()
                except Error as e:
                  cprint('-------------------------------','red')
                  print("Error while connecting to MySQL", e)
                  cprint('-------------------------------','red')
                  connection.rollback()
                  #pbar.next()
                  pbar.finish()
                  break
          except Error as e:
            cprint('-------------------------------','red')
            print("Error while connecting to MySQL", e)
            cprint('-------------------------------','red')
            break
          finally:
            if (connection.is_connected()):
              cursor.close()
              connection.close()
if (connection.is_connected()):
  cursor.close()
  connection.close()
cprint(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>\nDONE - FOREX imports complete\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'green')

Forex bhavcopy 2021-04-08_forex.csv create...SUCCESS

Connected to MySQL Server version 8.0.13-4


N/A% (0 of 48) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


Ingesting FOREX bhavcopy /2021-04-08_forex.csv into DB....: ('7FUybt81z6',)


100% (48 of 48) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


-------------------------------
Error while connecting to MySQL 1264 (22003): Out of range value for column 'trd_val' at row 1
-------------------------------
Forex bhavcopy 2021-04-09_forex.csv create...SUCCESS

Connected to MySQL Server version 8.0.13-4


...

N/A% (0 of 48) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


Ingesting FOREX bhavcopy /2021-04-12_forex.csv into DB....: ('7FUybt81z6',)


100% (48 of 48) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


-------------------------------
Error while connecting to MySQL 1264 (22003): Out of range value for column 'trd_val' at row 1
-------------------------------
2021-04-13 is a Market Holiday
2021-04-14 is a Market Holiday
Forex bhavcopy 2021-04-15_forex.csv create...SUCCESS


KeyboardInterrupt: ignored

In [ ]:
#********************************************************
# Ingest Banned symbols from zerodha excel sheet into DB#
#********************************************************
import pandas as pd
import requests as requests
from progressbar import ProgressBar
import mysql.connector
from mysql.connector import Error
import time

# Connect to the database
connection = mysql.connector.connect(host='johnny.heliohost.org',
                                                database='akini_algotrade',
                                                user='akini',
                                                password='Drink7up@home')
# create cursor
cursor=connection.cursor()
csvfile = requests.get('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/edit#gid=0')
print(csvfile.status_code)
# Insert DataFrame recrds one by one.
df=pd.read_csv('https://docs.google.com/spreadsheets/d/1ZTyh6GiHTwA1d-ApYdn5iCmRiBLZoAtwigS7VyLUk_Y/export?format=csv&gid=0', usecols=["Stocks not allowed for MIS"])
pbar = ProgressBar()
#truncate banned symbols table
sql_truncate_table = "truncate banned_symbols;"
cursor.execute(sql_truncate_table)
connection.commit()
time.sleep(10)
#Prepare SQL query to INSERT a record into the database.
for i,row in pbar(df.iterrows()):
  sql_banned_symbols = "INSERT INTO banned_symbols (symbol) VALUES ('%s');" % (row[0])
  try:
    #Execute the SQL command
    cursor.execute(sql_banned_symbols)
    #Commit your changes in the database
    connection.commit()
  except Error as e:
    cprint('-------------------------------','red')
    print("Error while connecting to MySQL", e)
    cprint('-------------------------------','red')
    connection.rollback()
    #pbar.next()
    pbar.finish()
    break
if (connection.is_connected()):
              cursor.close()
              connection.close()
cprint(">>>>>>>>>>\nDONE\n>>>>>>>>>>",'green')

In [ ]:
#********************************************
# download and insert sector indices symbols#
#********************************************
import requests, os, io, pandas as pd
from datetime import datetime
import csv
import glob
import mysql.connector
from mysql.connector import Error
from progressbar import ProgressBar
from termcolor import colored, cprint
import time

#https://honingds.com/blog/pandas-read_csv/#indexcol
#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
d, m, y = '%02d' % today.day, '%02d' % today.month, '%02d' % today.year #extract day , month and year from  he date

if not os.path.isdir(base+y+'/sectors'):#if there is no directory already present at the path with the year as a folder then create it
      os.mkdir(base+y+'/sectors')

#add a not condition to the below to skip over it.
if os.path.exists(base+y+'/sectors'):
  for CleanUp in glob.glob(base+y+'/sectors/*.*'): # list out the files
    os.remove(CleanUp) #remove files in the folder
  #download automobile csv
  indices_automobile = requests.get('https://archives.nseindia.com/content/indices/ind_niftyautolist.csv').content
  aut_csv = pd.read_csv(io.StringIO(indices_automobile.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  aut_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_automobile.csv', index=False)
  #download Bank csv
  indices_bank = requests.get('https://archives.nseindia.com/content/indices/ind_niftybanklist.csv').content
  sec_bank_csv = pd.read_csv(io.StringIO(indices_bank.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_bank_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_bank.csv', index=False)
  #download sec_consumer_durable csv
  indices_consumer_durable = requests.get('https://archives.nseindia.com/content/indices/ind_niftyconsumerdurableslist.csv').content
  sec_consumer_durable_csv = pd.read_csv(io.StringIO(indices_consumer_durable.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_consumer_durable_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_consumer_durable.csv', index=False)
  #download financial services csv
  indices_fin_serv = requests.get('https://archives.nseindia.com/content/indices/ind_niftyfinancelist.csv').content
  sec_fin_serv_csv = pd.read_csv(io.StringIO(indices_fin_serv.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_fin_serv_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_fin_serv.csv', index=False)
  #download fmcg csv
  indices_fmcg = requests.get('https://archives.nseindia.com/content/indices/ind_niftyfmcglist.csv').content
  sec_fmcg_csv = pd.read_csv(io.StringIO(indices_fmcg.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_fmcg_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_fmcg.csv', index=False)
  #download IT csv
  indices_it = requests.get('https://archives.nseindia.com/content/indices/ind_niftyitlist.csv').content
  sec_it_csv = pd.read_csv(io.StringIO(indices_it.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_it_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_it.csv', index=False)
  #download media csv
  indices_media = requests.get('https://archives.nseindia.com/content/indices/ind_niftymedialist.csv').content
  sec_media_csv = pd.read_csv(io.StringIO(indices_media.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_media_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_media.csv', index=False)
  #download metal csv
  indices_metal = requests.get('https://archives.nseindia.com/content/indices/ind_niftymetallist.csv').content
  sec_metal_csv = pd.read_csv(io.StringIO(indices_metal.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_metal_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_metal.csv', index=False)
  #download oil_gas csv
  indices_oil_gas = requests.get('https://www1.nseindia.com/content/indices/ind_niftyoilgaslist.csv').content
  sec_oil_gas_csv = pd.read_csv(io.StringIO(indices_oil_gas.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_oil_gas_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_oil_gas.csv', index=False)
  #download pharma csv
  indices_pharma = requests.get('https://archives.nseindia.com/content/indices/ind_niftypharmalist.csv').content
  sec_pharma_csv = pd.read_csv(io.StringIO(indices_pharma.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_pharma_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_pharma.csv', index=False)
  #download realty csv
  indices_realty = requests.get('https://archives.nseindia.com/content/indices/ind_niftyrealtylist.csv').content
  sec_realty_csv = pd.read_csv(io.StringIO(indices_realty.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
  sec_realty_csv.to_csv(base+y+'/sectors/'+y+'-'+m+'-'+d+'_realty.csv', index=False)

#truncate sector_indice_symbols table
sql_truncate_table = "truncate sector_indice_symbols;"
cursor.execute(sql_truncate_table)
connection.commit()
time.sleep(10)
#list out the files in the sector folder
for files in glob.glob(base+y+'/sectors/*.*'):
  print('\nIngesting sectoral CSV '+files)
  try:
    connection = mysql.connector.connect(host='johnny.heliohost.org',
                                        database='akini_algotrade',
                                        user='akini',
                                        password='Drink7up@home')
    
    db_Info = connection.get_server_info()
    print("\nConnected to MySQL Server version", db_Info)
    cursor = connection.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    cursor.fast_executemany = True
    with open(files, newline='',  encoding="utf8") as csvfile:
      csvdata = csv.reader(csvfile, escapechar="\'")
      #skip the 1st row as it will be header
      next(csvdata)
      pbar = ProgressBar()
      for row in pbar(list(csvdata)):
      #Prepare SQL query to INSERT a record into the database.
        sql= "INSERT INTO sector_indice_symbols (company_name,sector, symbol) \
        VALUES ('%s','%s','%s');" % (row[0], row[1], row[2])
        try:
          #Execute the SQL command
          cursor.execute(sql)
          #Commit your changes in the database
          connection.commit()
        except Error as e:
          cprint('-------------------------------','red')
          print("Error while connecting to MySQL", e)
          cprint('-------------------------------','red')
          connection.rollback()
          #pbar.next()
          pbar.finish()
          break
  except Error as e:
    cprint('-------------------------------','red')
    print("Error while connecting to MySQL", e)
    cprint('-------------------------------','red')
    break
cprint(">>>>>>>>>>\nDONE\n>>>>>>>>>>",'green')
if (connection.is_connected()):
              cursor.close()
              connection.close()

In [ ]:
#empty Folder
import pandas as pd
import glob, os,io

base = '/content/drive/My Drive/algotrade/2020'
for CleanUp in glob.glob(base+'/Futures/*.*'): # list out the files
#  os.remove(CleanUp) #remove files in the folder

####################
#Remote MySQl Creds#
####################
Username: 7FUybt81z6
Database name: 7FUybt81z6
Password: cvn1KKOijU
Server: remotemysql.com
Port: 3306

#################################################
#Kini's Out of control Strategy Stocks and FOREX#
#################################################
SELECT symbol, open, high, low, close, bhavcopydate, tottrdqty,deliverable, deliverable_percent
  FROM bhavcopy 
  where bhavcopydate >= date_sub(curdate(),interval 10 day)
  and close between 150 and 1000
  and symbol not in (select symbol from banned_symbols)
 
  SELECT symbol, max(high), min(low), stddev(high-low) stddev,avg((high-low)/open) as VFM, 
  avg(high-low),(avg(high-low)+stddev(high-low)) UCL, high-low, (avg(high-low)-stddev(high-low)) LCL
  FROM bhavcopy
  where bhavcopydate >= date_sub(curdate(),interval 10 day)
  #and symbol='ASAHISONG'
  group by symbol;
##################################################
#Kini's Out of control Strategy Forex #
SELECT symbol, open, high, low, close, exp_date, open_int, trd_qty, bhavcopydate
  FROM forex
  where bhavcopydate >= curdate()-10
  and exp_date between '2020-08-01' and '2020-09-01'

SELECT symbol, open high, low, close, bhavcopydate, stddev(high-low) stddev, variance(high-low) var, avg(high-low),(avg(high-low)+stddev(high-low)) UCL, high-low, (avg(high-low)-stddev(high-low)) LCL
  FROM forex 
  where bhavcopydate >= curdate()-10
  group by symbol
##################################################  


#############################################
#max upside or downside potential of a stock#
#############################################
SELECT symbol,AVG(uspot_array.uspot) as median_upside_potential_percent
FROM (
SELECT symbol,(high-open) as uspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM bhavcopy , (SELECT @rownum:=0) r
  WHERE symbol = 'geship'
  and 
  bhavcopydate >= date_sub(curdate(),interval 7 day)
  ORDER BY uspot
) as uspot_array
WHERE uspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) )

SELECT symbol,AVG(dspot_array.dspot) as median_downside_potential_percent
FROM (
SELECT symbol,(low-open) as dspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM bhavcopy , (SELECT @rownum:=0) r
  WHERE symbol='geship'
  and bhavcopydate >= date_sub(curdate(),interval 7 day)
  ORDER BY dspot
) as dspot_array
WHERE dspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

#############################################
#max upside or downside potential of forex#
#############################################
#############################################
#max upside or downside potential of forex#
#############################################
SELECT AVG(uspot_array.uspot) as median_upside_potential_in_Paisa
FROM (
SELECT (high-open) as uspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM forex , (SELECT @rownum:=0) r
  WHERE symbol='GBPINR' and exp_date between '2021-06-01' and '2021-07-01'
  and bhavcopydate >= date_sub(curdate(),interval 9 day)
  ORDER BY uspot
) as uspot_array
WHERE uspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

SELECT AVG(dspot_array.dspot) as median_downside_potential_in_Paisa
FROM (
SELECT (low-open) as dspot, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
  FROM forex , (SELECT @rownum:=0) r
  WHERE symbol='GBPINR'  and exp_date between '2021-06-01' and '2021-07-01'
  and bhavcopydate >= date_sub(curdate(),interval 9 day)
  ORDER BY dspot
) as dspot_array
WHERE dspot_array.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) );

#############################################
# volume growth                              #
#############################################
select symbol,bhavcopydate, if(@last_entry = 0, 0, round(((tottrdqty - @last_entry) / @last_entry) * 100,2)) "growth rate", @last_entry := tottrdqty from (select @last_entry := 0) x, (select symbol,tottrdqty, bhavcopydate,delivery_percent from bhavcopy where bhavcopydate >=now() - INTERVAL 7 day and symbol='acc') y;

select symbol,bhavcopydate, if(@last_entry = 0, 0, round(((trd_qty - @last_entry) / @last_entry) * 100,2)) "growth rate", @last_entry := trd_qty from (select @last_entry := 0) x, (select symbol,trd_qty, bhavcopydate from forex where bhavcopydate >= now() - INTERVAL 7 day and exp_date between '2021-08-01' and '2021-09-01' and symbol='GBPINR') y;

###########################################################  
# most volatile stocks in the database in the last 10 days#
###########################################################
SELECT symbol,CLOSE,`bhavcopydate`, VARIANCE(high-low) var 
  FROM bhavcopy 
  where `tottrdqty`> '1000000'
  and CLOSE between 50 and 200
  and `bhavcopydate` >= date_sub(curdate(),interval 10 day)
  and symbol in (select DISTINCT symbol from futures)
  GROUP BY symbol  
ORDER BY `var`  DESC

#######################################################################
#Max High and low in the last 5 trading sessions , they are KEY levels#
#######################################################################

select symbol, max(high), min(low) from bhavcopy
where bhavcopydate >= date_sub(curdate(),interval 7 day)
and symbol in ('PETRONET','GSPL','SBIN','TVSMOTOR','RBLBANK','TATASTEEL')
group by symbol

In [ ]:
//@version=4
study(shorttitle="AVCA MA+NR", title="2 Moving Average Color Direction Detection ", overlay=true)

// === INPUTS

ma_1 = input(true, title="Slow-MA")
ma_type   = input(defval="EMA", title="Slow-MA Type: ", options=["SMA", "EMA", "WMA", "VWMA", "SMMA", "DEMA", "TEMA", "HullMA", "ZEMA", "TMA", "SSMA"])
ma_len    = input(defval=25, title="Slow-MA Length", minval=1)
ma_src    = input(close, title="Slow-MA Source")
reaction_ma_1  = input(defval=3, title="Slow-MA Reaction", minval=1)

ma_2 = input(true, title="Fast-MA")
ma_type_b   = input(defval="EMA", title="Fast-MA Type: ", options=["SMA", "EMA", "WMA", "VWMA", "SMMA", "DEMA", "TEMA", "HullMA", "ZEMA", "TMA", "SSMA"])
ma_len_b    = input(defval=9, title="Fast-MA Length", minval=1)
ma_src_b    = input(close, title="Fast-MA Source")
reaction_ma_2  = input(defval=3, title="Fast-MA Reaction", minval=1)

filling = input(true, title="Shade the area between Fast and Slow EMA")

// SuperSmoother filter
// © 2013  John F. Ehlers
variant_supersmoother(src,len) =>
    a1 = exp(-1.414*3.14159 / len)
    b1 = 2*a1*cos(1.414*3.14159 / len)
    c2 = b1
    c3 = (-a1)*a1
    c1 = 1 - c2 - c3
    v9 = 0.0
    v9 := c1*(src + nz(src[1])) / 2 + c2*nz(v9[1]) + c3*nz(v9[2])
    v9
    
variant_smoothed(src,len) =>
    v5 = 0.0
    v5 := na(v5[1]) ? sma(src, len) : (v5[1] * (len - 1) + src) / len
    v5

variant_zerolagema(src,len) =>
    ema1 = ema(src, len)
    ema2 = ema(ema1, len)
    v10 = ema1+(ema1-ema2)
    v10
    
variant_doubleema(src,len) =>
    v2 = ema(src, len)
    v6 = 2 * v2 - ema(v2, len)
    v6

variant_tripleema(src,len) =>
    v2 = ema(src, len)
    v7 = 3 * (v2 - ema(v2, len)) + ema(ema(v2, len), len)             
    v7
    
variant(type, src, len) =>
    type=="EMA"     ? ema(src,len) : 
      type=="WMA"   ? wma(src,len): 
      type=="VWMA"  ? vwma(src,len) : 
      type=="SMMA"  ? variant_smoothed(src,len) : 
      type=="DEMA"  ? variant_doubleema(src,len): 
      type=="TEMA"  ? variant_tripleema(src,len): 
      type=="HullMA"? wma(2 * wma(src, len / 2) - wma(src, len), round(sqrt(len))) :
      type=="SSMA"  ? variant_supersmoother(src,len) : 
      type=="ZEMA"  ? variant_zerolagema(src,len) : 
      type=="TMA"   ? sma(sma(src,len),len) : sma(src,len)


// === Moving Average
ma_series = variant(ma_type,ma_src,ma_len)
ma_series_b = variant(ma_type_b,ma_src_b,ma_len_b)

direction = 0
direction := rising(ma_series,reaction_ma_1) ? 1 : falling(ma_series,reaction_ma_1) ? -1 : nz(direction[1])
change_direction= change(direction,1)

direction_b = 0
direction_b := rising(ma_series_b,reaction_ma_2) ? 1 : falling(ma_series_b,reaction_ma_2) ? -1 : nz(direction_b[1])
change_direction_b= change(direction_b,1)

// Plot MA series and color it according too direction
pcol = direction>0 ? color.green : direction<0 ? color.red : na
a=plot( timeframe.isdaily and ma_1? ma_series :na, color=pcol,join=true,linewidth=2,title="Slow-MA Plot")

pcol_b = direction_b>0 ? color.green : direction_b<0 ? color.red : na
b=plot( timeframe.isdaily and ma_2? ma_series_b :na, color=pcol_b,join=true,linewidth=1,title="Fast-MA Plot")

fill(a,b,color=direction==1 and direction_b==1 and filling == true?color.green:direction==-1 and direction_b==-1and filling == true?color.red:direction==1 and direction_b==-1and filling == true?color.new(color.purple,70):direction==-1 and direction_b==1and filling == true?color.new(color.purple,70):color.white)

/////// Alerts ///////

alertcondition(change_direction,title="Change Direction Slow-MA",message="Change Direction Slow-MA")
alertcondition(change_direction_b,title="Change Direction Fast-MA",message="Change Direction Fast-MA")

alertcondition(crossunder(ma_series_b,ma_series),title="Cross Under Fast-MA/Slow-MA",message="Cross Under Fast-MA/Slow-MA")
alertcondition(crossunder(ma_series,ma_series_b),title="Cross Under Slow-MA/Fast-MA",message="Cross Under Slow-MA/Fast-MA")

alertcondition(crossover(ma_series_b,ma_series),title="Cross Over Fast-MA/Slow-MA",message="Cross Over Fast-MA/Slow-MA")
alertcondition(crossover(ma_series,ma_series_b),title="Cross Over Slow-MA/Fast-MA",message="Cross Over Slow-MA/Fast-MA")

alertcondition(cross(ma_series_b,ma_series),title="Cross Slow-MA/Fast-MA",message="Cross Slow-MA/Fast-MA")

//PLOT NR7
tOpen = security(syminfo.tickerid, timeframe.period, open, barmerge.gaps_off, barmerge.lookahead_on)
tHigh = security(syminfo.tickerid, timeframe.period, high, barmerge.gaps_off, barmerge.lookahead_on)
tLow = security(syminfo.tickerid, timeframe.period, low, barmerge.gaps_off, barmerge.lookahead_on)
tClose = security(syminfo.tickerid, timeframe.period, close, barmerge.gaps_off, barmerge.lookahead_on)


enableNR = input(false, title="Show Narrow Range (NR) 4, 7, 21 ?")
enableNRIB = input(true, title="Show Narrow Range (NR) 4, 7, 21 + Inside Day ?", tooltip="NR 4, 7, 21 + Inside Bars ?")

// Narrow Range 21
length_NR21 = 21
NR_Check(range) =>
    range < lowest(range, length_NR21 - 1)[1] ? true : false

range_NR_Prep = tHigh - tLow
range_NR = tHigh[1] - tLow[1]
range_isCurrentLowest = NR_Check(range_NR_Prep)
nr21ib = range_isCurrentLowest and high<=high[1] and low>=low[1]

// NR 4 and NR 7
nr4 = (tr <= tr[1]) and (tr <= tr[2]) and (tr <= tr[3])
nr7 = nr4 and (tr <= tr[4]) and (tr <= tr[5]) and (tr <= tr[6])

//NR 4 and NR 7 + Inside Bars
nr4ib = (tr <= tr[1]) and (tr <= tr[2]) and (tr <= tr[3])
     and (high<=high[1] and low>=low[1])

nr7ib = nr4ib and (tr <= tr[4]) and (tr <= tr[5]) and (tr <= tr[6])
     and (high<=high[1] and low>=low[1])

barcolor(enableNR and range_isCurrentLowest? color.black : na, title= "Narrow Range 21", offset = 0, editable = true)
barcolor(enableNR and nr7? color.orange : na, title= "Narrow range 7", offset = 0, editable = true)
barcolor(enableNR and nr4? color.yellow : na, title= "Narrow range 4", offset = 0, editable = true)

plotshape(enableNRIB and nr4ib and timeframe.isdaily, title= "NR 4 + Inside Bars", location=location.belowbar, color=color.yellow, style=shape.labelup,text='NR4+IC', textcolor=color.black, size=size.small)
plotshape(enableNRIB and nr7ib and timeframe.isdaily, title= "NR 7 + Inside Bars", location=location.belowbar, color=color.orange, style=shape.labelup,text='NR7+IC', textcolor=color.black, size=size.small)
plotshape(enableNRIB and nr21ib and timeframe.isdaily, title= "NR 21 + Inside Bars", location=location.abovebar, style = shape.labeldown, color =color.black ,text='NR21+IC', textcolor=color.white, size=size.small)

// Colored background
timeinrange(res, sess) => time(res, sess) != 0
premarket = #FFD68A
regular = #BDBF00
postmarket = #FFD68A
color notrading = na
sessioncolor = timeinrange("30", "0230-0330") ? premarket :
  timeinrange("30", "0330-1130") ? regular :
  timeinrange("30", "1130-1530") ? postmarket : notrading
bgcolor(sessioncolor, transp=75)

//phigh, plow
useColors = input(title="Fill with colors?",defval=true)
isess = session.regular
t = tickerid(syminfo.prefix, syminfo.ticker, session=isess)
igaps = barmerge.gaps_off
yesterdayHigh = security(t,"D",high[1],gaps=igaps, lookahead=barmerge.lookahead_on)
yesterdayLow = security(t,"D",low[1],gaps=igaps, lookahead=barmerge.lookahead_on)

// Plot the other time frame's data
a_hl=plot(timeframe.isintraday ? yesterdayHigh : na, color=color.lime, linewidth=1, style=plot.style_linebr)
b_hl=plot(timeframe.isintraday ? yesterdayLow : na, color=color.red, linewidth=1, style=plot.style_linebr)
fill(a_hl,b_hl,color=(useColors? close>yesterdayHigh ? color.green : (close < yesterdayLow ? color.red : color.blue) : color.white)  )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')